In [1]:
from agent import Agent
from functions import *
import sys
from environment import SimpleTradeEnv
from random import randint
import numpy as np
import matplotlib.pyplot as plt
#import ga
import pickle
import timeit
import re

In [2]:
def plot_values(S,E,I,R,x,beds):
    plt.figure(figsize=(10,5.5))
    plt.grid()
    plt.title('Active Cases for Different Values of Average Rate of Spread')
    plt.xlabel('Number of Days after the initial 3 Cases')
    plt.ylabel('Number of Cases')
    #plt.plot(120000+np.zeros(1000), 'black', linewidth = 3, label = 'healthcare capacity')
    
    #plt.plot(x, S, '-b', label='Sus')
    plt.plot(x, E, '-g', label='Exp')    
    plt.plot(x, I, '-r', label='Inf')   
    plt.axhline(y=beds, color='b', linestyle='-', label="Total beds")
    #plt.plot(x, R, '-y', label='Rec')    
    plt.legend()
    plt.show()
def calculate_state(data, R):
    aux = data.copy()
    aux.append(R)    
    return aux[1:]

In [3]:
# Define parameters Covid-19
n_days = 120
dt = 1 # Interval of times
t = np.linspace(0, n_days, int(n_days/dt) + 1)
pop = 8918653 # 50M
exp =   5000 # 200 muertos x 95%
init_vals = 1 - exp/pop, exp/pop, 0, 0  # Susceptible, Exposed, Infected, Recovered.
#init_vals = pop-inf, inf*2, inf, 0  # Susceptible, Exposed, Infected, Recovered.
alpha = 0.192307
beta  = 0.055
gamma = 0.0686

rho = 1
params = alpha, beta, gamma, rho

ICU = 0.05 # Percentage of infected people that needs beds
n_beds = 0.0015 # percentage of beds in mexico 1.5 per 1000
r0,r1,r2,r3  = 1,0.78,0.67,0.61 # r4 represets reward for exceeding
b_0,b_1,b_2,b_3 = 2.2,0.348,0.24,0.055
Mills_per_day = 3348


# Phase 0: No economic reduction
# Phase 1: 22% reduction
# Phase 2: 33% reduction
# Phase 3: 39% reduction

In [4]:
def listToString(s):  
    # initialize an empty string 
    str1 = ""  
    # traverse in the string   
    for ele in s:  
        str1 += str(ele)   
    # return string   
    return str1

In [8]:
# Define parameters agent
window_size = 25
initial_data = window_size*[0] 
n_episodes = 1000
batch_size = 200
decay_rate = 0.99
random = []
solutions, time_solutions = [], []
Best_global_Sequence = []
Best_global_value = 0

for experiments in range(1,1000):
    agent = Agent(window_size,eps_d=decay_rate)
    done = True
    max_reward = -100000000
    start = timeit.default_timer()
    
    # Review the state of the model
    for e in range(n_episodes):
        phase = 3 # The initial phase starts in 3, if the agent predicts 1 it will change to phase 2
                           # if the predicted action is 0, it will remain in 3.
        seq_actions = []
        total_rewards = 0
        economy = 0
        S_0, E_0, I_0, R_0 = init_vals
        S, E, I, R = [S_0], [E_0], [I_0], [R_0]
        alpha, beta, gamma, rho = params
        dt = t[1] - t[0]
        state = calculate_state(initial_data,I_0)
        printed = True

        last_action = -1
        tmp = 1 # Temperature
        n_beds = 0.0015
        
        
        for _ in t[1:]:
            action = agent.act(np.array([state]),e)
            if action == 1 and phase != 0: # We move to another phase when the action 
                phase = phase - 1          # is 1 and the phase is different to 0.
                
    
            #action = randint(0, 2) # when playing random actions
            seq_actions.append(phase)
            
            # Susceptible, Exposed, Infected, Recovered.
            next_S = S[-1] - (beta*S[-1]*I[-1])*dt
            next_E = E[-1] + (beta*S[-1]*I[-1] - alpha*E[-1])*dt
            next_I = (I[-1] + (alpha*E[-1] - gamma*I[-1])*dt)
            next_R = R[-1] + (gamma*I[-1])*dt                      
            S.append(next_S)
            E.append(next_E)
            I.append(next_I)
            R.append(next_R)

            if len(agent.memory) > batch_size and randint(0, 10) == 1:  # Retrain the neural network
                agent.Replay()
                
            if phase == 0: # Impose total isolation (People work from home)
                beta = b_0   # rho reduces the contact rate
                reward = r0 * Mills_per_day
            if phase == 1: # Impose partial confinement
                beta = b_1
                reward = r1 * Mills_per_day
            if phase == 2: # Maintain 2 meters social distance and washing hands
                beta = b_2
                reward = r2 * Mills_per_day
            if phase == 3: # Do nothing
                beta = b_3
                reward = r3 * Mills_per_day            

            economy += reward
            #if economia > (0.8*120*Mills_per_day):
            #    reward = 0
            if (next_I*ICU) > n_beds: # if the number of beds is higher then reward is zero
                reward = 0 # Otherwise we increase by 1000                
            total_rewards += reward
            
            next_state = calculate_state(state,next_I)            
            agent.memory.append((np.array([state]), action, reward, np.array([next_state]), done))
            state = next_state

        stop = timeit.default_timer()
        agent.memory.append((np.array([state]), action, reward, np.array([next_state]), False))
        res = np.stack([S, E, I, R]).T
        agent.decrease_epsilon()       
        total_rewards = round(total_rewards)
        
        if max_reward < total_rewards:
            max_reward = total_rewards
            max_seq_actions = seq_actions
            #plot_values(res[:,0],res[:,1],res[:,2],res[:,3],t,n_beds)
            # save the model to disk
            #print("Saving")
            #filename = 'finalized_model.sav'
            #pickle.dump(agent, open(filename, 'wb'))
            print("Episode " + str(e) + "/" + str(n_episodes) + "/" + str(experiments) \
                  + " - " + str(total_rewards) + "/" + str(max_reward))            
            print("Best: ",   max_reward, ", Actions:  (0)", round(seq_actions.count(0)/len(seq_actions),2),
                "(1)", round(seq_actions.count(1)/len(seq_actions),2), " ,(2)", round(seq_actions.count(2)/len(seq_actions),2), 
                "(3)", round(seq_actions.count(3)/len(seq_actions),2))
            print("Best solution: ", max_reward, " : ", max_seq_actions)
        #else:
            #print("Reading")
            #agent = pickle.load(open('finalized_model.sav', 'rb'))
        if e % 100 == 0:
            print("Episode " + str(e) + "/" + str(n_episodes) + "/" + str(experiments) \
                  + " - " + str(total_rewards) + "/" + str(max_reward))
            print("Best: ",   max_reward, ", Actions:  (0)", round(seq_actions.count(0)/len(seq_actions),2),
                "(1)", round(seq_actions.count(1)/len(seq_actions),2), " ,(2)", round(seq_actions.count(2)/len(seq_actions),2), 
                "(3)", round(seq_actions.count(3)/len(seq_actions),2)) 

    if Best_global_value < max_reward:
        Best_global_value = max_reward
        Best_global_Sequence = max_seq_actions
    solutions.append(max_reward)
    time_solutions.append(round(stop - start,2))
    print('Time: ', round(stop - start,2), "seg \n")
    print("Solutions: ", solutions)
    print("Best solution: ", Best_global_value, " : ", Best_global_Sequence)

action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0

action:  1
action:  1
action:  0
action:  1
action:  1
action:  1
action:  0
action:  0
action:  1
action:  0
action:  0
action:  1
action:  0
action:  1
action:  1
action:  0
action:  0
action:  1
action:  0
action:  0
action:  0
action:  0
action:  1
action:  0
action:  1
action:  1
action:  1
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  1
action:  1
action:  1
action:  1
action:  1
action:  1
action:  0
action:  1
action:  0
action:  1
action:  0
action:  0
action:  1
action:  0
action:  0
action:  1
action:  1
action:  1
action:  1
action:  1
action:  0
action:  0
action:  1
action:  1
action:  0
action:  1
action:  0
action:  1
action:  0
action:  1
action:  1
action:  0
action:  1
action:  1
action:  0
action:  0
action:  1
action:  0
action:  0
action:  0
action:  0
action:  0
action:  1
action:  0
action:  0
action:  1
action:  0
action:  0
action:  0
action:  1
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0
action:  0

KeyboardInterrupt: 

In [ ]:
print("Solutions: ", solutions)
print("Best solution: ", Best_global_value, " : ", Best_global_Sequence)
print("Time solutions: ", time_solutions)

In [ ]:
Best solution:  0  :  0
Time solutions:  [154.95, 152.85, 148.13, 145.12, 146.38, 145.2, 143.19, 144.44, 137.21, 135.1, 136.62, 135.55, 133.65, 136.78, 134.16, 138.63, 134.85, 137.67, 137.34, 136.08, 134.88, 135.01, 136.08, 133.5, 134.99, 135.47, 134.89, 135.54, 137.7, 136.24, 134.71, 146.23, 159.89]

In [ ]:
Solutions =  [-11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492, -11684492]
new = []
for i in Solutions:
    new.append(i+(120*100000))
print(new)